In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import seaborn as sns
import sys
import os
import io 

import pypandoc
import os
import sys

project_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
if project_root not in sys.path:
    sys.path.append(project_root)


In [ ]:
def rst_to_plain_text_pandoc(input_path, output_dir):
    base_name = os.path.splitext(os.path.basename(input_path))[0]
    output_file = os.path.join(output_dir, f"{base_name}.txt")

    try:

        output_text = pypandoc.convert_file(
            input_path,
            'plain',  # 출력 포맷: plain text
            format='rst',  # 입력 포맷: rst
            extra_args=['--wrap=none'] 
        )


        with open(output_file, 'w', encoding='utf-8') as f:
            f.write(output_text)

    except FileNotFoundError:
        print(f"  [Error] File not found: {input_path}")
    except OSError as e:
        print(f"  [Error] Pandoc OS Error processing {os.path.basename(input_path)}: {e}")
        print("        Is 'pandoc' installed and in your system PATH?")
    except Exception as e:
        print(f"  [Error] Pandoc conversion failed for {os.path.basename(input_path)}: {e}")


def process_directory_with_pandoc(input_dir, output_dir):

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for filename in os.listdir(input_dir):
        if filename.endswith('.rst'):
            input_path = os.path.join(input_dir, filename)
            print(f"Processing: {input_path}")
            rst_to_plain_text_pandoc(input_path, output_dir)




In [ ]:
#실행
input_dir = '../Docs/Pandas_doc_233/user_guide'
output_dir = '../chunk_pandoc' # 출력 폴더 구분
process_directory_with_pandoc(input_dir, output_dir)

In [ ]:
import chromadb
import os
import uuid 

client = chromadb.Client()


try:
    from chromadb.utils import embedding_functions
    embedding_func = embedding_functions.DefaultEmbeddingFunction()
    print("ChromaDB 기본 임베딩 함수를 사용합니다 (테스트용). 한국어 모델 사용을 권장합니다.")

except ImportError:
    embedding_func = None 
except Exception as e:
    print(f"임베딩 함수 로딩 중 오류 발생: {e}")
    embedding_func = None

collection_name = "pandas_docs_chunked"

try:
    collection = client.get_or_create_collection(
        name=collection_name,
        embedding_function=embedding_func, # 여기서 임베딩 함수 지정
         metadata={"hnsw:space": "cosine"} # 코사인 유사도
    )
    print(f"ChromaDB 컬렉션 '{collection_name}' 준비 완료.")
except Exception as e:
    print(f"컬렉션 생성/가져오기 실패: {e}")
    collection = None


chunk_dir = '../chunk_pandoc' 
documents = []
metadatas = []
ids = []

if collection and embedding_func:
    try:
        for filename in os.listdir(chunk_dir):
            if filename.endswith('.txt'):
                file_path = os.path.join(chunk_dir, filename)
                try:
                    with open(file_path, 'r', encoding='utf-8') as f:
                        content = f.read()
                        if content.strip(): # 내용이 비어있지 않은 경우만 추가
                            documents.append(content)
                            # 메타데이터에 파일 이름 추가 (다른 정보 추가 가능)
                            metadatas.append({"source": filename})
                            # 각 문서에 고유 ID 부여
                            ids.append(str(uuid.uuid4()))
                        else:
                            print(f"  [Info] Skipping empty file: {filename}")

                except Exception as e:
                    print(f"  [Error] Failed to read or process file {filename}: {e}")


        if documents:
            try:

                collection.add(
                    documents=documents,
                    metadatas=metadatas,
                    ids=ids
                )

                count = collection.count()
                print(f"현재 컬렉션 '{collection_name}'의 문서 수: {count}")

            except Exception as e:
                print(f"ChromaDB에 데이터 추가 중 오류 발생: {e}")
        else:
            print("추가할 문서가 없습니다.")

    except FileNotFoundError:
        print(f"청크 디렉토리를 찾을 수 없음 - {chunk_dir}")
    except Exception as e:
        print(f"디렉토리 처리 중 오류 발생: {e}")
else:
    if not embedding_func:
        print("인베딩 모듈 준비 안됨")
    if not collection:
         print("ChromaDB 설정이 안됨")




In [ ]:
import chromadb
import os

client = chromadb.Client()

try:

    from chromadb.utils import embedding_functions
    embedding_func = embedding_functions.DefaultEmbeddingFunction()

except ImportError:

    embedding_func = None
except Exception as e:
    print(f"임베딩 함수 로딩 중 오류 발생: {e}")
    embedding_func = None

collection_name = "pandas_docs_chunked" # 이전에 생성한 컬렉션 이름

if embedding_func:
    try:
        collection = client.get_collection(
            name=collection_name,
            embedding_function=embedding_func
        )
        print(f"ChromaDB 컬렉션 '{collection_name}' 가져오기 완료.")
        print(f"현재 컬렉션의 문서 수: {collection.count()}")

        query_text = "pd.merge()"
        print(f"\n쿼리: \"{query_text}\"")

        n_results_to_fetch = 3 # 가져올 문서 개수
        
        results = collection.query(
            query_texts=[query_text],       # 쿼리 텍스트 리스트
            n_results=n_results_to_fetch,   # 반환할 결과 수
            include=['documents', 'metadatas', 'distances'] 
        )


        print(f"\n--- 상위 {n_results_to_fetch}개 유사 문서 결과 ---")

        if results and results.get('ids') and results['ids'][0]:
            for i in range(len(results['ids'][0])):
                doc_id = results['ids'][0][i]
                distance = results['distances'][0][i]
                metadata = results['metadatas'][0][i]
                document = results['documents'][0][i]

                print(f"\n[결과 {i+1}]")
                print(f"  ID: {doc_id}")
                print(f"  Distance: {distance:.4f}")
                print(f"  Metadata (Source): {metadata.get('source', 'N/A')}")
                # print(f"  Document: {document[:200]}...") # 내용 미리보기 


        else:
            print("유사한 문서를 찾지 못했습니다.")

    except Exception as e:
        print(f"ChromaDB 쿼리 중 오류 발생: {e}")
        # 컬렉션이 존재하지 않는 경우 등의 오류 처리
        if "does not exist" in str(e):
             print(f"컬렉션 '{collection_name}'이(가) 존재하지 않습니다. 데이터 추가 코드를 먼저 실행했는지 확인")

else:
    print("임베딩 함수 오류")